## 01 IO, Dimensional Reduction, and Clustering


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Author: [Antonia Mey -- @ppxasjsm](https://github.com/ppxasjsm)

## Learning objectives:
- Load a molecular trajectory
- Extract different features from the trajectory, such as backbone torsions or heavy atom positions
- Cluster trajectory data using k-means or regular spatial clustering
- Understand what the differences between TICA, PCA and VAMP are for dimensionality reduction methods and how to cluster on reduced spaces. 

You will be using the following functions in pyemma:

- `pyemma.coordinates.featurizer()` to define a selection of features we want to extract,
- `pyemma.coordinates.load()` to load data into memory, and
- `pyemma.coordinates.source()` to create a streamed feature reader in case the data does not fit into memory.
- `pyemma.plots.plot_feature_histograms()` to show the distributions of all loaded features,
- `pyemma.plots.plot_density()` to visualize the sample density, and
- `pyemma.plots.plot_free_energy()` to visualize the free energy surface of two selected features.
- `pyemma.coordinates.pca()` to perform a principal components analysis,
- `pyemma.coordinates.tica()` to perform a time-lagged independent component analysis, and
- `pyemma.coordinates.vamp()` to analyze the quality of some feature spaces, perform dimension reduction, and
- `pyemma.coordinates.cluster_kmeans()` to perform a $k$-means clustering, and
- `pyemma.coordinates.cluster_regspace()` to perform a regspace clustering.



**Reading time**:
~ 30 mins

**Jupyter cheat sheet**:
- to run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- to get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- you can find the full documentation at [PyEMMA.org](http://www.pyemma.org).

## Table of Contents
1. [Working with MD trajectories](#MD)    
   1.1 [Loading trajectories](#load)   
   1.2 [Selecting features](#feat)   
   1.3 [Exercises](#exerc1)   
2. [Dimensionality reduction and clustering](#dtraj)   
   2.1 [Clustering](#clust)   
   2.2 [Saving objects](#save)   
   2.3 [Discrete trajectories](#disc)
   2.4 [PCA, TICA, VAMP](#dim)   
   2.5 [Exercises](#exerc2)   
3. [Advanced exercises](#exerc3)   

## 1. Working with MD trajectories
<a id="MD"></a>

In this section you will learn how to load traejctories with pyemma and how to extract common features such as backbone dihedral angles from these trajectories

#### Let's get all the necessary imports out of the way

In [ ]:
%pylab inline
import mdshare
import pyemma
# for visualization of molecular structures:
import nglview
import mdtraj
from threading import Timer
from nglview.player import TrajectoryPlayer
import seaborn as sbn
sbn.set_context("paper",font_scale=1.4)

### 1.1. Loading a molecular trajectory in `*.xtc` format (alanine dipeptide)
<a id="load"></a>

To load molecular dynamics data from one of the standard file formats e.g ( `*.xtc`),
we need not only the actual simulation data, but a topology file, too.

In [ ]:
pdb = mdshare.fetch('alanine-dipeptide-nowater.pdb', working_directory='data')
files = mdshare.fetch('alanine-dipeptide-*-250ns-nowater.xtc', working_directory='data')
print(pdb)
print(files)

If you just want to read `xyz` coordinates of your trajectories you can use the load function.

⚠️The warning about **plain coordinates** is triggered,
because these coordinates will include diffusion as a dynamical process,
which might not be what one is interested in.
If the molecule of interest has been aligned to a reference prior the analysis,
it is fine to use these coordinates, but we will see that there are better choices. 

In [ ]:
data = pyemma.coordinates.load(files, top=pdb)

We can visualise the structure using NGLview:

In [ ]:
widget = nglview.show_mdtraj(mdtraj.load(pdb))
p = TrajectoryPlayer(widget)
widget.add_ball_and_stick()
p.spin = True
def stop_spin():
    p.spin = False
    widget.close()
Timer(30, stop_spin).start()
widget

### 1.2. Selecting features
<a id="feat"></a>

In PyEMMA, the `featurizer` is a central object that incorporates the system's topology.
We start by creating it using the topology file.
Features are now easily computed by adding the target feature.
If no feature is added, the featurizer will extract Cartesian coordinates.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)

Now we pass the featurizer to the load function to extract the Cartesian coordinates from the trajectory files into memory.
⚠️ For real world examples it is best to use the `source()` function,
because not all your simulation data may fit into your workstations memory. 

⚠️The warning about **plain coordinates** is triggered,
because these coordinates will include diffusion as a dynamical process,
which might not be what one is interested in.
If the molecule of interest has been aligned to a reference prior the analysis,
it is fine to use these coordinates, but we will see that there are better choices. 

In [ ]:
data = pyemma.coordinates.load(files, features=feat)
print('type of data:', type(data))
print('lengths: %d' %len(data))
print('shape of elements: ', data[0].shape)
print('n_atoms: %d' %feat.topology.n_atoms)

Next, we start adding features which we want to extract from the simulation data.
Here, we want to load the backbone torsions,
because these angles are known to describe all flexibility in the system.
Since this feature is two dimensional, it is also easier to visualize.
Please note that, in complex systems, it is not trivial to visualize plain input features.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(periodic=False)

⚠️ Please note that the trajectories have been aligned to a reference structure before.
Since in this case we loose track of the periodic box,
we have to switch off the `periodic` flag for the torsion angle computations.
By default PyEMMA assumes your simulation uses periodic boundary conditions.

We can always call the featurizer's `describe()` method to show which features are requested.
You might have noticed that you can combine arbitrary features by having multiple calls to `add_` methods of the featurizer.

In [ ]:
data = pyemma.coordinates.load(files, features=feat)

print('type of data:', type(data))
print('lengths:', len(data))
print('shape of elements:', data[0].shape)

After we have selected all desired features,
we can call the `load()` function to load all features into memory or,
alternatively, the `source()` function to create a streamed feature reader.
For now, we will use `load()`:

In [ ]:
print(feat.describe())

#### Visualsing data
Apparently, we have loaded a list of three two-dimensional `numpy.ndarray` objects from our three trajectory files.
We can visualize these features using the aforementioned plotting functions,
but to do so we have to concatenate the three individual trajectories:

In [ ]:
data_concatenated = np.concatenate(data)
pyemma.plots.plot_feature_histograms(data_concatenated, feature_labels=feat);

We now use PyEMMA's `plot_density()` and `plot_free_energy()` functions to create Ramachandran plots of our system:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)
# the * operator used in a function call is used to unpack
# the iterable variable into its single elements. 
pyemma.plots.plot_density(*data_concatenated.T, ax=axes[0])
pyemma.plots.plot_free_energy(*data_concatenated.T, ax=axes[1], legacy=True)
for ax in axes.flat:
    ax.set_xlabel('$\phi$')
    ax.set_aspect('equal')
axes[0].set_ylabel('$\psi$')
fig.tight_layout()

⚠️ Please note that these functions visualise the density and free energy of the sampled data, not the equilibrium distribution of the underlying system. To account for non-equiblibrium data, you can supply frame-wise weights using the weights parameter, which will be covered later. 

However, you can clearly see different basins which will be used for the MSM construction later. 

#### Heavy atoms feature
Let us look at a different featurization example and load the positions of all heavy atoms instead.
We create a new featurizer object and use its `add_selection()` method to request the positions of a given selection of atoms.
For this selection, we can use the `select_Heavy()` method which returns the indices of all heavy atoms.

Again, we load the data into memory and show what we loaded using the `describe()` method:

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Heavy())

data = pyemma.coordinates.load(files, features=feat)

feat.describe()

⚠️ Please note that PyEMMA has flattened the $x, y$ and $z$ coordinates into an array that will be used for further analysis.

We visualize the distributions of all loaded features:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat, ax=ax)
fig.tight_layout()

#### `load()` versus `source()`

Using `load()`, we put the full data into memory.
This is possible for all examples in this tutorial.

Many real world applications, though, require more memory than your workstation might provide.
For these cases, you should use the `source()` function:

In [ ]:
reader = pyemma.coordinates.source(files, features=feat)
print(reader)

This function creates a reader, wich allows to stream the underlying data in chunks instead of the full set.
Most of the functions in the `coordinates` sub-package accept data in memory as well as readers.
However, some plotting functions require the data to be in memory.
To load a (sub-sampled) subset into memory, we can use the `get_output()` method with a stride parameter:

In [ ]:
data_output = reader.get_output(stride=5)
len(data_output)
print('number of frames in first file: {}'.format(reader.trajectory_length(0)))
print('number of frames after striding: {}'.format(len(data_output[0])))

We now have loaded every fifth frame into memory.
Again, we can visualize the (concatenated) features with a feature histogram:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(
    np.concatenate(data_output), feature_labels=feat, ax=ax)
fig.tight_layout()

### 1.3. Exercises on loading data and extracting features
<a id='exerc1'></a>

The exercises are announced by the keyword **Exercise** and followed by an incomplete cell.
Missing parts are indicated by
```python
#FIXME
```


#### Exercise 1.3.1: Heavy atom distances

Please fix the following code block such that the distances between all heavy atoms are loaded and visualized.

**Hint**: try to use the auto-complete feature on the feat object to gain some insight.
Also take a look at the previous demonstrations.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
pairs = feat.pairs(# FIXME)
feat. #FIXME

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(
    np.concatenate(data), feature_labels=feat, ax=ax)
fig.tight_layout()

#### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
pairs = feat.pairs(feat.select_Heavy())
feat.add_distances(pairs)

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(
    np.concatenate(data), feature_labels=feat, ax=ax)

fig.tight_layout()

#### Exercise 1.3.2: Write out heavy atom distances

Please fix the following code to write out heavy atom distances to file and load them again

**Hint**: try to use the auto-complete feature on the feat object to gain some insight.

In [ ]:
feat. #FIXME

#### Solution

In [ ]:
feat.save('test.obj', overwrite=True)
heavy_atoms = pyemma.load('test.obj')
print(heavy_atoms)

#### Exercise 1.3.3: Torsion angles again...

Please fix the following code to compute the `sin` and `cos` of the backbone torsional angles, as well as plot these features?



In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

print(feat.describe())

In [ ]:
data = pyemma.coordinates.#FIXME
data_concatenated = #FIXME
pyemma.plots.#FIXME
fig.tight_layout()

#### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(cossin=True, periodic=False)

print(feat.describe())

Finally, we visualize the (concatenated) features:

In [ ]:
data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)
fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(data_concatenated, feature_labels=feat, ax=ax)
fig.tight_layout()

#### Exercise 1.3.4

Complete the following code block to load/visualize the position of all backbone atoms.

**Hint**: You might find the `select_Backbone()` method of the featurizer object helpful.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

# Concatanate the data
#FIXME

# Plot the data
#FIXME

#### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Backbone())

data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)

fig, ax = plt.subplots(figsize=(10, 12))
pyemma.plots.plot_feature_histograms(data_concatenated, feature_labels=feat, ax=ax)
fig.tight_layout()

## 2. Dimensionality Reduction and Clustering
<a id="dtraj"></a>

In this section we will look at how we can obtain a discrete trajectory from the features of the molecular trajectory. 

### 2.1. Clustering
<a id="clust"></a>
Let's go back to the nice backbone torsion view and use these to cluster data. 

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(periodic=False)
data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)
print(feat.describe())

We will be looking at two clsutering methods [kmeans](https://jakevdp.github.io/PythonDataScienceHandbook/05.11-k-means.html) and 'regular spatial'.  

With only 2 torsional angles, we can directly attempt to discretize or cluster the data,
e.g., with $k$-means with $100$ centers and a stride of $10$ to reduce the computational effort.
In real world examples we also might encounter low dimensional feature spaces
which do not require further dimension reduction techniques to be clustered efficiently.

#### k-means

In [ ]:
cluster_kmeans = pyemma.coordinates.cluster_kmeans(data, k=100, max_iter=50, stride=10, n_jobs=2)

... or with a regspace technique where all centers should have a minimal pairwise distance of $0.5$ units of length.

#### regular spacial

In [ ]:
cluster_regspace = pyemma.coordinates.cluster_regspace(data, dmin=0.3,  n_jobs=2)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=True)
for ax, cls in zip(axes.flat, [cluster_kmeans, cluster_regspace]):
    pyemma.plots.plot_density(*data_concatenated.T, ax=ax, cbar=False, alpha=0.1, logscale=True)
    ax.scatter(*cls.clustercenters.T, s=15, c='C1')
    ax.set_xlabel('$\phi$')
    ax.set_ylabel('$\psi$')
fig.tight_layout()

Have you noticed how the $k$-means centers follow the density of the data points while the regspace centers are spread uniformly over the whole area? 

If your are only interested in well sampled states, you should use a density based method to discretize.
If exploring new states is one of your objectives,
it might be of advantage to place states also in rarely observed regions.
The latter is especially useful in adaptive sampling approaches,
because in the initial phase you want to explore the phase space as much as possible.
The downside of placing states in areas of low density is that we will have poor statistics on these states. 

Another advantage of regular space clustering is that it is fast in comparison to $k$-means:
regspace clustering runs in linear time while $k$-means is superpolynomial in time.

⚠️ For large datasets we also offer a mini batch version of $k$-means which has the same semantics as the original method but trains the centers on subsets of your data.
This tutorial does not cover this case, but you should keep in mind that $k$-means requires your low dimensional space to fit into your main memory.


### 2.2. Saving and restoring the clustering object
<a id="save"></a>

You noticed how the clustering even with the relatively small dataset took a substantial amount of time it would be good to not having to rerun the clustering every time and be able to save this information. 

PyEMMA provides a convenience method for saving these objects. 
Just try it out:

In [ ]:
cluster_kmeans.save('kmeans.pyemma', model_name='ala_kmeans_100', overwrite=True)
cluster_regspace.save('regspatial.pyemma', model_name='ala_regspace', overwrite=True)

Now we have stored the current state of the clustering estimator to disk.
A file can contain multiple models, this is why we have used the `model_name` argument to specify the name.
If omitted, the estimator will be saved under the name `default_model`.

Assume that we have restarted our Python session and do not want to re-compute everything.
We can now restore the previously saved estimator via

In [ ]:
cluster_restored = pyemma.load('kmeans.pyemma', model_name='ala_kmeans_100')

# check that nothing has changed
np.testing.assert_allclose(cluster_restored.clustercenters, cluster_kmeans.clustercenters, atol=1e-15)

To check the contents of a file, you can utilize the list_models function of PyEMMA:

In [ ]:
pyemma.list_models('kmeans.pyemma')

### 2.3. Discrete trajectory generation
<a id="disc"></a>
The main result of a discretization for Markov modeling, however,
is not the set of centers but the time series of discrete states.
These are accessible via the `dtrajs` attribute of any clustering object:

In [ ]:
print(cluster_kmeans.dtrajs)
print(cluster_regspace.dtrajs)

For each trajectory passed to the clustering object, we get a corresponding discrete trajectory.

Please note that as an alternative to clustering algorithms such as $k$-means and regspace,
it is possible to manually assign the data to cluster centers using the `pyemma.coordinates.assign_to_centers()` function.

### 2.4. PCA, TICA, VAMP 
<a id="#dim"></a>

Instead of discretizing the full (two-dimensional) space, we can attempt to find a one-dimensional subspace which
1. describes the slow dynamics of the data set equally well but
2. is easier to discretize.

One widespread method for dimension reduction is [principal component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) which finds a subspace with maximized variance. In pyemma you can use it in this way:

In [ ]:
pca = pyemma.coordinates.pca(data, dim=1)
pca_output = pca.get_output()
print(pca_output)

Another technique is the time-lagged independent component analysis (TICA) which finds a subspace with maximized autocorrelation <a id="ref-1" href="#cite-tica2">molgedey-94</a>, <a id="ref-2" href="#cite-tica">perez-hernandez-13</a>.
To compute the autocorrelation, we need a time shifted version of the data.
This time shift is specified by the `lag` argument.
For the current example, we choose a lag time of $1$ step.

In [ ]:
tica = pyemma.coordinates.tica(data, dim=1, lag=1)
tica_output = tica.get_output()
print(tica_output)

Instead of TICA, we can also employ the variational approach for Markov processes (VAMP) to obtain a coordinate transform <a id="ref-3" href="#cite-vamp-preprint">wu-17</a>.
In contrast to TICA, VAMP can be applied to non-equilibrium / non-reversible data.

In [ ]:
vamp = pyemma.coordinates.vamp(data, dim=1, lag=1)
vamp_output = vamp.get_output()
print(vamp_output)

⚠️ While there are many cases where PCA can find a suitable subspace,
there are also many cases where the PCA-based subspace neglects the slow dynamics.

In our example, the slow process is the jump along the $\phi$ coordinate. For all three methods, we show the distribution after projecting the full dynamics onto a one-dimensional subspace (left) and the direction of projection (right).

In [ ]:
pca_concatenated = pca_output[0]
tica_concatenated = tica_output[0]
vamp_concatenated = vamp_output[0]

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    np.concatenate([pca_concatenated, tica_concatenated, vamp_concatenated], axis=1),
    feature_labels=['PCA', 'TICA', 'VAMP'],
    ax=axes[0])
pyemma.plots.plot_density(*data_concatenated.T,ax=axes[1], cbar=False, alpha=0.1, logscale=True)
axes[1].plot(
    [0, 3 * pca.eigenvectors[0, 0]],
    [0, 3 * pca.eigenvectors[1, 0]],
    linewidth=3,
    label='PCA')
axes[1].plot(
    [0, 3 * tica.eigenvectors[0, 0]],
    [0, 3 * tica.eigenvectors[1, 0]],
    linewidth=3,
    label='TICA')
axes[1].plot(
    [0, 3 * vamp.singular_vectors_right[0, 0]],
    [0, 3 * vamp.singular_vectors_right[1, 0]],
    linewidth=3,
    label='VAMP')
axes[1].set_xlabel('$\phi$')
axes[1].set_ylabel('$\psi$')
#axes[1].set_xlim(-4, 4)
#axes[1].set_ylim(-4, 4)
#axes[1].set_aspect('equal')
axes[1].legend()
fig.tight_layout()

We see that TICA and VAMP project along the $\phi$-axis and, thus, yield a subspace which clearly resolves both metastable states.
PCA on the other hand projects closely along the $\psi$-axis and resolves a metastable state between the $\alpha$-helix and $\beta$-sheet space in the Ramachandran plot, but not along to the L-$\alpha$ minimum.
This is a case in point where variance maximization does not find a subspace which resolves the slowest dynamics of the system.

This effect can also be seen when we plot the subspace time series:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
ax.plot(pca_concatenated[:300], label='PCA')
ax.plot(tica_concatenated[:300], label='TICA')
# note that for better comparability, we enforce the same direction as TICA
ax.plot(vamp_concatenated[:300] * -1, label='VAMP')
ax.set_xlabel('time / steps')
ax.set_ylabel('feature values')
ax.legend()
fig.tight_layout()

For all TICA/VAMP and PCA, we observe two different sets of metastable states. PCA resolves metastability along $\psi$ a and VAMP and TICA along $\phi$. In the analysis later on it will become clear that this is in fact the coordinate for the slowest dynamics of the system. 

In many applications, however, we also need to understand what our coordinate transform means in physical terms.
This, in general, might be less obvious.
Hence, it might be instructive to inspect the correlations of features to the independent components:

In [ ]:
fig, ax = plt.subplots()
i = ax.imshow(tica.feature_TIC_correlation, cmap='bwr', vmin=-1, vmax=1)

ax.set_xticks([0])
ax.set_xlabel('IC')

ax.set_yticks([0, 1])
ax.set_ylabel('input feature')

fig.colorbar(i);

In this simple example, we clearly see a significant correlation between the $\phi$ component of the input data and the first independent component.

⚠️ In practice you almost never would like to use PCA as dimension reduction method in MSM building,
as it does not preserve kinetic variance. We are showing it here in these exercises to make this point clear.

In [ ]:
cluster = pyemma.coordinates.cluster_kmeans(tica, k=10, stride=5) # use only 1/5 of the input data to find centers

We can now just obtain the discrete trajectories by accessing the property on the cluster instance.
This will get all the TICA projected trajectories and assign them to the centers computed on the reduced data set.

In [ ]:
dtrajs = cluster.dtrajs
print('Assignment:', dtrajs)
dtrajs_len = [len(d) for d in dtrajs]
for dtraj_len, input_len in zip(dtrajs_len, reader.trajectory_lengths()):
    print('Input length:', input_len, '\tdtraj length:', dtraj_len)

### 2.5. Exercises
<a id='exerc2'></a>

#### Exercise 2.5.1: Data loading and using PCA on that dataset

Load the heavy atoms' positions into memory. Discretizing a $30$-dimensional feature space is impractical.
Let's use PCA to find a low-dimensional projection and visualize the marginal distributions of all principal components (PCs) as well as the joint distributions for the first two PCs:

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME
data = pyemma.coordinates.load(#FIXME)

print('We have {} features.'.format(feat.dimension()))

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat, ax=ax)
fig.tight_layout()

# PCA dimensionality reduction
pca = #FIXME
pca_concatenated = np.concatenate(#FIXME)

fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharex=True)
pyemma.plots.plot_feature_histograms(
    pca_concatenated, ['PC {}'.format(i + 1) for i in range(pca.dimension())], ax=axes[0])
pyemma.plots.plot_density(*pca_concatenated[:, :2].T, ax=axes[1], cbar=False, logscale=True)
pyemma.plots.plot_free_energy(*pca_concatenated[:, :2].T, ax=axes[2], legacy=False)
for ax in axes.flat[1:]:
    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')
fig.tight_layout()

#### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Heavy())
data = pyemma.coordinates.load(files, features=feat)

print('We have {} features.'.format(feat.dimension()))

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat, ax=ax)
fig.tight_layout()

# PCA dimensionality reduction
pca = pyemma.coordinates.pca(data)
pca_concatenated = np.concatenate(pca.get_output())

fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharex=True)
pyemma.plots.plot_feature_histograms(
    pca_concatenated, ['PC {}'.format(i + 1) for i in range(pca.dimension())], ax=axes[0])
pyemma.plots.plot_density(*pca_concatenated[:, :2].T, ax=axes[1], cbar=False, logscale=True)
pyemma.plots.plot_free_energy(*pca_concatenated[:, :2].T, ax=axes[2], legacy=False)
for ax in axes.flat[1:]:
    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')
fig.tight_layout()

With the default parameters, PCA will return as many dimensions as necessary to explain $95\%$ of the variance;
in this case, we have found a five-dimensional subspace which does seem to resolve some metastability in the first three principal components.

#### Exercise 2.5.2: TICA visualization

Apply TICA and visualize the marginal distributions of all independent components (ICs) as well as the joint distributions of the first two ICs.

In [ ]:
tica =  #FIXME
tica_concatenated = np.concatenate(tica.get_output())

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(
    tica_concatenated, ['IC {}'.format(i + 1) for i in range(tica.dimension())], ax=axes[0])
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], cbar=False, logscale=True)
pyemma.plots.plot_free_energy(*tica_concatenated[:, :2].T, ax=axes[2], legacy=False)
for ax in axes.flat[1:]:
    ax.set_xlabel('IC 1')
    ax.set_ylabel('IC 2')
fig.tight_layout()

#### Solution

In [ ]:
tica = pyemma.coordinates.tica(data)
tica_concatenated = np.concatenate(tica.get_output())

fig, axes = plt.subplots(1, 3, figsize=(12, 3))
pyemma.plots.plot_feature_histograms(
    tica_concatenated, ['IC {}'.format(i + 1) for i in range(tica.dimension())], ax=axes[0])
pyemma.plots.plot_density(*tica_concatenated[:, :2].T, ax=axes[1], cbar=False, logscale=True)
pyemma.plots.plot_free_energy(*tica_concatenated[:, :2].T, ax=axes[2], legacy=False)
for ax in axes.flat[1:]:
    ax.set_xlabel('IC 1')
    ax.set_ylabel('IC 2')
fig.tight_layout()

#### 2.5.3 Exercise: TICA correlations

TICA, by default, uses a lag time of $10$ steps, kinetic mapping and a kinetic variance cutoff of $95\%$ to determine the number of ICs.
We observe that this projection does resolve some metastability in both ICs.
Whether these projections are suitable for building Markov state models, though, remains to be seen in later on.

As we discussed in the first example, the physical meaning of the TICA projection is not directly clear.
We can analyze the feature TIC correlation as we did above:

In [ ]:
fig, ax = plt.subplots(figsize=(3, 8))
i = ax.imshow(#FIXME, cmap='bwr')

ax.set_xticks(range(tica.dimension()))
ax.set_xlabel('IC')

ax.set_yticks(range(feat.dimension()))
ax.set_yticklabels(feat.describe())
ax.set_ylabel('input feature')

fig.colorbar(i);

#### Solution

In [ ]:
fig, ax = plt.subplots(figsize=(3, 8))
i = ax.imshow(tica.feature_TIC_correlation, cmap='bwr')

ax.set_xticks(range(tica.dimension()))
ax.set_xlabel('IC')

ax.set_yticks(range(feat.dimension()))
ax.set_yticklabels(feat.describe())
ax.set_ylabel('input feature')

fig.colorbar(i);

This is not helpful as it only shows that some of our $x, y, z$-coordinates correlate with the TICA components.
Since we rather expect the slow processes to happen in backbone torsion space, this comes to no surprise. 

To understand what the TICs really mean, let us do a more systematic approach and scan through some angular features.
We add some randomly chosen angles between heavy atoms and the backbone angles that we already know to be a good feature:

In [ ]:
feat_test = pyemma.coordinates.featurizer(pdb)
feat_test.add_backbone_torsions(periodic=False)
feat_test.add_angles(feat_test.select_Heavy()[:-1].reshape(3, 3), periodic=False)
data_test = pyemma.coordinates.load(files, features=feat_test)
data_test_concatenated = np.concatenate(data_test)

For the sake of simplicity, we use scipy's implementation of Pearson's correlation coefficient which we compute between our test features and TICA projected $x, y, z$-coordinates:

In [ ]:
from scipy.stats import pearsonr
test_feature_TIC_correlation = np.zeros((feat_test.dimension(), tica.dimension()))

for i in range(feat_test.dimension()):
    for j in range(tica.dimension()):
        test_feature_TIC_correlation[i, j] = pearsonr(
            data_test_concatenated[:, i],
            tica_concatenated[:, j])[0]

In [ ]:
vm = abs(test_feature_TIC_correlation).max()

fig, ax = plt.subplots()
i = ax.imshow(test_feature_TIC_correlation, vmin=-vm, vmax=vm, cmap='bwr')

ax.set_xticks(range(tica.dimension()))
ax.set_xlabel('IC')

ax.set_yticks(range(feat_test.dimension()))
ax.set_yticklabels(feat_test.describe())
ax.set_ylabel('input feature')

fig.colorbar(i);

From this simple analysis, we find that features that correlated most with our TICA projection are indeed the backbone torsion angles used previously.
We might thus expect the dynamics in TICA space to be similar to the one in backbone torsion space.

⚠️ Please note that in general, we do not know which feature would be a good observable.
Thus, a realistic scenario might require a much broader scan of a large set of different features.

However, it should be mentioned that TICA projections do not necessarily have a simple physical interpretation.
The above analysis might very well end with feature TIC correlations that show no significant contributor and rather hint towards a complicated linear combination of input features.


#### Exercise 2.5.4: PCA parameters

Perform PCA on the heavy atoms' positions data set with a target dimension of two;
then discretize the two-dimensional subspace using $k$-means with $100$ centers and a stride of $15$ to reduce the computational effort.

**Hint:** Look up the parameters of `pyemma.coordinates.pca()`, especially the `dim` parameter.

In [ ]:
pca =  # FIXME
pca_concatenated =  # FIXME

cluster = pyemma.coordinates.cluster_kmeans(pca, k=100, max_iter=50, stride=15, n_jobs=2)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    pca_concatenated, ['PC {}'.format(i + 1) for i in range(pca.dimension())], ax=axes[0])
pyemma.plots.plot_density(*pca_concatenated.T, ax=axes[1], cbar=False, alpha=0.1, logscale=True)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('PC 1')
axes[1].set_ylabel('PC 2')
fig.tight_layout()

#### Solution

In [ ]:
pca = pyemma.coordinates.pca(data, dim=2)
pca_concatenated = np.concatenate(pca.get_output())

cluster = pyemma.coordinates.cluster_kmeans(pca, k=100, max_iter=50, stride=15, n_jobs=2)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    pca_concatenated, ['PC {}'.format(i + 1) for i in range(pca.dimension())], ax=axes[0])
pyemma.plots.plot_density(*pca_concatenated.T, ax=axes[1], cbar=False, alpha=0.1, logscale=True)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('PC 1')
axes[1].set_ylabel('PC 2')
fig.tight_layout()

#### Exercise 2.5.5: TICA parameters

Perform TICA at lag time $1$ step on the heavy atoms' positions data set with a target dimension of two;
then discretize the two-dimensional subspace using $k$-means with $100$ centers and a stride of $5$ to reduce the computational effort.

**Hint:** Look up the parameters of `pyemma.coordinates.tica()`, especially the `dim` and `lag` parameters.

In [ ]:
tica =  # FIXME
tica_concatenated =  # FIXME

cluster =  # FIXME

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    tica_concatenated, ['IC {}'.format(i + 1) for i in range(tica.dimension())], ax=axes[0])
pyemma.plots.plot_density(*tica_concatenated.T, ax=axes[1], cbar=False, alpha=0.1, logscale=True)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
fig.tight_layout()

#### Solution

In [ ]:
tica = pyemma.coordinates.tica(data, lag=1, dim=2)
tica_concatenated = np.concatenate(tica.get_output())

cluster = pyemma.coordinates.cluster_kmeans(tica, k=100, max_iter=50, stride=1, n_jobs=2)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
pyemma.plots.plot_feature_histograms(
    tica_concatenated, ['IC {}'.format(i + 1) for i in range(tica.dimension())], ax=axes[0])
pyemma.plots.plot_density(*tica_concatenated.T, ax=axes[1], cbar=False, alpha=0.1, logscale=True)
axes[1].scatter(*cluster.clustercenters.T, s=15, c='C1')
axes[1].set_xlabel('IC 1')
axes[1].set_ylabel('IC 2')
fig.tight_layout()

Have you noticed the difference in the first two ICs for lag times $10$ steps vs. $1$ step (e.g., result of exercises `2.5.2` and `2.5.3`)?

## 3. More Exercises
<a id="exerc3"></a>
Only try this exercise, if you have run out of time. 

Can you run the same analysis as you have shown in exercise 2.3.4, but using VAMP instead? Plot the ICs with the input features and the density plot

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Heavy())
data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)
vamp = #FIXME

#### Solution

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Heavy())
data = pyemma.coordinates.load(files, features=feat)
data_concatenated = np.concatenate(data)
vamp = pyemma.coordinates.vamp(data_concatenated, lag=20, dim=3)
vamp_concatenated = np.concatenate(vamp.get_output(stride=5))

cluster = pyemma.coordinates.cluster_kmeans(vamp, k=100, max_iter=50, stride=15,n_jobs=2)

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
pyemma.plots.plot_feature_histograms(
    tica_concatenated, ['IC {}'.format(i + 1) for i in range(tica.dimension())], ax=axes[0, 0])
for ax, (i, j) in zip(axes.flat[1:], [[0, 1], [1, 2], [0, 2]]):
    pyemma.plots.plot_density(*vamp_concatenated[:, [i, j]].T, ax=ax, cbar=False, alpha=0.1)
    ax.scatter(*cluster.clustercenters[:, [i, j]].T, s=15, c='C1')
    ax.set_xlabel('IC {}'.format(i + 1))
    ax.set_ylabel('IC {}'.format(j + 1))
fig.tight_layout()

## References


<a id="cite-tica2"/><sup><a href=#ref-1>[^]</a></sup>Molgedey, L. and Schuster, H. G.. 1994. _Separation of a mixture of independent signals using time delayed correlations_. [URL](http://dx.doi.org/10.1103/PhysRevLett.72.3634)

<a id="cite-tica"/><sup><a href=#ref-2>[^]</a></sup>Guillermo Pérez-Hernández and Fabian Paul and Toni Giorgino and Gianni De Fabritiis and Frank Noé. 2013. _Identification of slow molecular order parameters for Markov model construction_. [URL](https://doi.org/10.1063/1.4811489)

<a id="cite-vamp-preprint"/><sup><a href=#ref-3>[^]</a></sup>Wu, H. and Noé, F.. 2017. _Variational approach for learning Markov processes from time series data_. [URL](https://arxiv.org/pdf/1707.04659.pdf)

<a id="cite-aggarwal_surprising_2001"/><sup><a href=#ref-4>[^]</a></sup>Aggarwal, Charu C. and Hinneburg, Alexander and Keim, Daniel A.. 2001. _On the Surprising Behavior of Distance Metrics in High Dimensional Space_.




#### Disclaimer: 
This tutorial has been adapted from pyemma tutorials 01 and 02 (https://github.com/markovmodel/pyemma_tutorials)


<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Maintainers of the original notebooks [@cwehmeyer](https://github.com/cwehmeyer), [@marscher](https://github.com/marscher), [@thempel](https://github.com/thempel), [@psolsson](https://github.com/psolsson)

